# Analyze downloaded bird sound files 

## Introduction
The program analyze the bird song dataset downloaded with the file "AM - Download dataset.py" and checks the metadata itself.
Each folder saved have at least one file inside: json with metadata. 

> JSON is short for JavaScript Object Notation, and is a way to store information in an organized, easy-to-access manner. In a nutshell, it gives us a human-readable collection of data that we can access in a really logical manner. (source: https://www.copterlabs.com/json-what-it-is-how-it-works-how-to-use-it/)

If the algorithm found at least one file under searched term, those sound files are saved in the folder.

## Requirements
You should have installed followed libraries:
* statistics, matplotlib, json and numpy
* pandas - an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools
* mutagen - mutagen is a Python module to handle audio metadata
* librosa -  python package for music and audio analysis
* noisereduce - noise reduction in python using spectral gating

## Scirpt
Script in the notebook returns:
- [x] Number of files for a given bird, 
- [x] Minimum, maximum, and average length of a file, 
- [ ] Number of sounds with more than 1 tag,
- [x] Number of sounds with specified quality (e.g. none - 460, A- 102 recordings, B - 230 recordigns, ...).

Next, it analyze the recordings and:
- [x] Calculate the signal to noise ratio -> display a summary plot for all recordings, 
- [x] Shows 3 random bird-songs spectograms, melgrams and sound waves, 
- [x] Test denoising of 3 random recordings.



In [ ]:
import json
import pandas as pd

In [ ]:
from AM_downloadDataset import read_data

In [ ]:
countries = ['Poland', 'Germany', 'Slovakia', 'Czech', 'Lithuania']

# initialize dictionary of a bird
bird = {
        'gen':'Parus',
        'spec':'major',
        'country':'',
        'number of files': 
            {
            'total' : 0,
            'quality':{'A':0,
                        'B':0,
                        'C':0,
                        'D':0,
                        'E':0,
                        'F':0,
                      }
            },
        'length':{'min':0,
                      'max':0,
                      'mean':0,
                      'median':0
                      }
        }



In [ ]:
from mutagen.mp3 import MP3
from statistics import mean, median

audioLengths = list()
audioLengthTemp = []
for country in range(len(countries)):
        # find the driectory with recordings
        bird['country']=bird['country']+' ' + countries[country]
        path = '../data/xeno-canto-dataset/' + bird['gen'] + ' '+bird['spec'] +' cnt '+ countries[country] + ' type song'
        print('Loading data from folder ' + path)
        
        # load info about the quality of the recording from json file
        qualityData = read_data('q', path)
        bird['number of files']['total']=bird['number of files']['total']+len(qualityData)
        for quality in bird['number of files']['quality']:
             bird['number of files']['quality'][quality]=bird['number of files']['quality'][quality]+qualityData.count(quality)
        
        # load MP3 file of every recording and check the length of a file
        idData = read_data('id', path)
        for audioFile in range(len(idData)):
            audioLengthTemp.append(MP3(path+'/'+bird['gen']+idData[audioFile]+'.mp3').info.length) 
                        
        audioLengths = list(audioLengthTemp) + list(audioLengths)

bird['length']['max']=max(audioLengths)
bird['length']['min']=min(audioLengths)
bird['length']['mean']=mean(audioLengths) 
bird['length']['median']=median(audioLengths) 


## Randomly test few recordings
In this section we randomly select a few (for exampke 3) recordings from a dataset and listen to it. Then we plot wavegrams, spectograms and melgrams.

In [ ]:
from random import sample

# find the driectory with recordings
path = '../data/xeno-canto-dataset/' + bird['gen'] + ' '+bird['spec'] + ' cnt '+ 'Poland' + ' type song'
print('Loading data from folder ' + path)

# load json file:  read all id numbers
idData = read_data('id', path)
qualityData = read_data('q',path)

# select random 5 recordings from Poland
randFiles = sample(range(len(idData)), 3)
print('Selected random files number:',randFiles)

In [ ]:
import IPython.display as ipd

for audioFile in randFiles:
    # path of random file
    filePath = path+'/'+bird['gen']+idData[audioFile]+'.mp3'
    print('Play the file number '+ str(audioFile)+', quality: '+ qualityData[audioFile])
    # show the recording and allow to play it
    ipd.display(ipd.Audio(filePath))


In [ ]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np


for audioFile in randFiles:
    # path of random file
    filePath = path+'/'+bird['gen']+idData[audioFile]+'.mp3'

    # plot recording signal
    y, sr = librosa.load(filePath,duration=10)
    plt.figure(figsize=(10, 4))
    librosa.display.waveplot(y, sr=sr)
    plt.title('Monophonic - file number '+str(audioFile))
    plt.show()
    
    # plot spectogram
    plt.figure(figsize=(10, 4))
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    librosa.display.specshow(D, y_axis='linear')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Linear-frequency power spectrogram - file number '+str(audioFile))
    plt.show()


In [ ]:
for audioFile in randFiles:
    # path of random file
    filePath = path+'/'+bird['gen']+idData[audioFile]+'.mp3'

    # loading files again
    y, sr = librosa.load(filePath,duration=10)

    # plot mel-spectogram
    # Passing through arguments to the Mel filters
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.power_to_db(S,ref=np.max),
                          y_axis='mel', fmax=8000,x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram - file number '+str(audioFile))
    plt.tight_layout()
    plt.show()

## Denoising
Testing denoising with the library **noisereduce** https://pypi.org/project/noisereduce/
#### Noise reduction in python using spectral gating
This algorithm is based (but not completely reproducing) on the one outlined by Audacity for the noise reduction effect
The algorithm requires two inputs:
* A noise audio clip comtaining prototypical noise of the audio clip - in the file below I fed the algirthm with the whole recording as the recording with a noise
* A signal audio clip containing the signal and the noise intended to be removed


In [ ]:
import noisereduce as nr
for audioFile in randFiles:
    # path of random file
    filePath = path+'/'+bird['gen']+idData[audioFile]+'.mp3'
    print('Before denoising: File number '+ str(audioFile)+', quality: '+ qualityData[audioFile])
    # show the recording and allow to play it
    ipd.display(ipd.Audio(filePath))
    
    # load a file again
    data, r = librosa.load(filePath,duration=10)
    
    # plot mel-spectogram before denosing
    S = librosa.feature.melspectrogram(y=data, sr=sr, n_mels=128,fmax=8000)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.power_to_db(S,ref=np.max),
                          y_axis='mel', fmax=8000,x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram - file number '+str(audioFile))
    plt.tight_layout()
    plt.show()    
    
    # reduce noise
    reduced_noise = nr.reduce_noise(audio_clip=data, noise_clip=data, prop_decrease = 1, verbose=False)
    
    # plot mel-spectogram after denosing
    print("After denoising")
    ipd.display(ipd.Audio(reduced_noise,rate=r))
    S = librosa.feature.melspectrogram(y=reduced_noise, sr=sr, n_mels=128,fmax=8000)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.power_to_db(S,ref=np.max),
                          y_axis='mel', fmax=8000,x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram - file number '+str(audioFile))
    plt.tight_layout()
    plt.show()


## General statistics summary
Printing summarized information about selected bird.
Then, to summarize the statistics we plot:
* Plot - length of recordings with the line of a median and average length
* Histogram - to plot histogram we will find number of bins auntomaticially. The selected method is ‘fd’ (Freedman Diaconis Estimator) - Robust (resilient to outliers) estimator that takes into account data variability and data size

In [ ]:
print('******',bird["gen"],' ',bird['spec'],'******',)
print("Found ", bird['number of files']['total']," files in total, from countires: ", bird['country'])
print("Number of files with quality: A = ", bird['number of files']['quality']['A'],
          ", B = ", bird['number of files']['quality']['B'],
         ", C = ", bird['number of files']['quality']['C'],
         ", D = ", bird['number of files']['quality']['D'],
         ", E = ", bird['number of files']['quality']['E'])
print("Shortest file: ", bird['length']['min'], ' seconds')
print("Longest file: ", bird['length']['max'], ' seconds')
print("Average length: ", bird['length']['mean'], 's, ',"\nmedian length: ", bird['length']['median'], 's' )

# Plot a length of recordings with the line of a median and average length
plt.figure(figsize=(10, 4))
plt.title('The recording length [s]')
plt.ylabel('Recording length (s)')
plt.xlabel('File number')
plt.plot(audioLengths)
plt.axhline(y=bird['length']['median']+20, color='r', linestyle='-') # plot the line at a median
plt.axhline(y=bird['length']['mean'], color='y', linestyle='-') # plot the line at a mean

# Plot histogram
plt.figure(figsize=(10, 4))
plt.title('Histogram of recordings length [s]')
plt.xlabel('Recording length (s)')
plt.ylabel('Number of files in the bin')
plt.hist(audioLengths, bins  = 'fd')
plt.show()

In [ ]:
import sys
from tqdm import tqdm
from time import sleep

print('\nThis may take some time...\n')

snr = list()
for country in range(len(countries)):
        # find the driectory with recordings
        bird['country']=bird['country']+' ' + countries[country]
        path = '../data/xeno-canto-dataset/' + bird['gen'] + ' '+bird['spec'] +' cnt '+ countries[country] + ' type song'
        print('Loading data from folder ' + path)

        idData = read_data('id', path)
        with tqdm(total=len(idData), file=sys.stdout) as pbar:
            for audioFile in range(len(idData)):
                pbar.set_description('Loading data from folder %d' %audioFile)
                pbar.update(1)
                data, r = librosa.load(path+'/'+bird['gen']+idData[audioFile]+'.mp3',duration=10)
                reduced_noise = nr.reduce_noise(audio_clip=data, noise_clip=data, prop_decrease = 1, verbose=False)
                signal2noise=reduced_noise/(data-reduced_noise)
                snr.append(median(signal2noise))

plt.figure(figsize=(10, 4))
plt.title('Signal to noise ratio for each file')
plt.plot(snr)
plt.show()

plt.figure(figsize=(10, 4))
plt.title('Histogram: signal to noise ratio for each file')
plt.hist(snr, bins  = 'fd')
plt.show()